In [90]:
import pandas as pd 
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyOAuth 
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

In [91]:
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [92]:
liked_songs = []
results = sp.current_user_saved_tracks()
while results:
    for item in results['items']:
        track = item['track']
        song_id = track['id']
        row = {"song_id": song_id}
        liked_songs.append(row)
    results = sp.next(results)
output = pd.DataFrame(liked_songs)

In [ ]:
#gather data using soptify API
song_ana = pd.DataFrame()
for i in range (len(output)) :
    analysis = sp.audio_features(output.at[i, "song_id"])
    analy = pd.DataFrame(analysis)
    song_ana = pd.concat([song_ana, analy])
song_ana = song_ana.drop(["type", "duration_ms", "time_signature", "mode"], axis= 1).reset_index().drop(["index"], axis= 1)
song_ana

In [94]:

# Normalize the DataFrame to be between 0 and 1
min_value = song_ana[["tempo"]].min()
max_value = song_ana[["tempo"]].max()
song_ana["tempo"] = (song_ana[["tempo"]] - min_value) / (max_value - min_value)

min_value = song_ana[["loudness"]].min()
max_value = song_ana[["loudness"]].max()
song_ana["loudness"] = (song_ana[["loudness"]] - min_value) / (max_value - min_value)

min_value = song_ana[["key"]].min()
max_value = song_ana[["key"]].max()
song_ana["key"] = (song_ana[["key"]] - min_value) / (max_value - min_value)


*starting to classifed*

In [95]:
#target cloumn
column_select = song_ana[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [96]:
song_ana["prediciton"] = loaded_model.predict(column_select)

In [97]:
calm = song_ana[song_ana['prediciton'] == 0 ]
happy = song_ana[song_ana['prediciton'] == 1 ]
energy = song_ana[song_ana['prediciton'] == 2 ]
sad = song_ana[song_ana['prediciton'] == 3 ]

In [98]:
len(calm)

42

In [99]:
len(happy)

90

In [100]:
len(energy)

595

In [101]:
len(sad)

948